# Clustering Crypto

In [551]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [525]:
# Use the following endpoint to fetch json data
url = 'https://min-api.cryptocompare.com/data/all/coinlist'

In [526]:
first_response = requests.get(url)
response_list = first_response.json()
response_list.values

<function dict.values>

In [527]:
import json
r = requests.get(url, data={'key':'value'})
print(r)


<Response [200]>


In [528]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

crypto_df = json.dumps(url)
crypto_df


'"https://min-api.cryptocompare.com/data/all/coinlist"'

In [529]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df=pd.read_csv('crypto_data.csv')

### Data Preprocessing

In [530]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'


In [531]:
# Keep only cryptocurrencies that are trading
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [532]:
# Keep only cryptocurrencies with a working algorithm


In [533]:
# Remove the "IsTrading" column
crypto=crypto_df.drop(columns='IsTrading')
crypto.head()


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [534]:
# Remove rows with at least 1 null value
Crypto_clean=crypto.dropna()
Crypto_clean.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [535]:
# Remove rows with cryptocurrencies having no coins mined
index_namesMine= Crypto_clean[Crypto_clean['TotalCoinsMined'] == 0].index
Crypto_clean.drop(index_namesMine, inplace = True)
Crypto_clean.head()

/Users/manoloserrano/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [536]:
# Drop rows where there are 'N/A' text values


In [537]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName=Crypto_clean['CoinName']

In [538]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
Crypto_noCoinName = Crypto_clean.drop(columns='CoinName')
Crypto_noCoinName.head()

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0


In [539]:
# Create dummy variables for text features
CryptoDummies=pd.get_dummies(Crypto_noCoinName)
CryptoDummies.head()

,TotalCoinsMined,Unnamed: 0_1337,Unnamed: 0_1CR,Unnamed: 0_404,Unnamed: 0_42,Unnamed: 0_8BIT,Unnamed: 0_AAC,Unnamed: 0_ABJ,Unnamed: 0_ABS,Unnamed: 0_ABY,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
0,4.199995e+01,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.055185e+09,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.927942e+10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [563]:
# Standardize data
#CryptoScaled=StandardScaler(CryptoDummies)
CryptoScaled=StandardScaler().fit_transform(CryptoDummies)
print(CryptoScaled[0:5])


[[-0.11416167 -0.04163054 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.09006124 -0.04163054 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [ 0.55458069 24.0208243  -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.11375222 -0.04163054 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.11170216 -0.04163054 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]]


### Reducing Dimensions Using PCA

In [564]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=2)

Crypto_pca = pca.fit_transform(CryptoScaled)

In [565]:
# Create a DataFrame with the principal components data
df_Crypto_pca = pd.DataFrame(
    data=Crypto_pca, columns=["principal component 1", "principal component 2"]
)
df_Crypto_pca.head()

,principal component 1,principal component 2
0,-0.375121,-0.083969
1,-0.334744,-0.158883
2,0.547754,-0.264012
3,-0.218248,0.007145
4,0.058980,0.158930


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [566]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_Crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k', y='inertia', xticks=k, title='Elbow Cure')

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [572]:
# Initialize the K-Means model

model = KMeans(n_clusters=3, random_state=0)

# Fit the model

model.fit(Crypto_pca)

# Predict clusters

predictions = model.predict(df_Crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features

df_Crypto_pca['class'] = model.labels_
df_Crypto_pca.head()

,principal component 1,principal component 2,class
0,-0.375121,-0.083969,0
1,-0.334744,-0.158883,0
2,0.547754,-0.264012,0
3,-0.218248,0.007145,0
4,0.058980,0.158930,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [573]:
# Create a 3D-Scatter with the PCA data and the clusters
df_Crypto_pca.hvplot.scatter(
    x='principal component 1',
    y='principal component 2',
    hover_cols=['class'],
    by='class',
)

:NdOverlay   [class]
   :Scatter   [principal component 1]   (principal component 2,class)

#### Table of Tradable Cryptocurrencies

In [546]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
